<a href="https://colab.research.google.com/github/inamsatti/ElectricVehicles/blob/main/Electric_car_price_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install kaggle

**Importing Libriries**

In [2]:
import kagglehub
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import missingno as msno
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score
from sklearn.preprocessing import LabelEncoder
import xgboost as xgb
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
import pickle
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
import torch
import torch.nn as nn
import torch.nn.functional as F


**Dataset loading**

In [ ]:
path = kagglehub.dataset_download("kkhandekar/cheapest-electric-cars/versions/1")
print("Path to dataset files:", path)
df1 = pd.read_csv("/root/.cache/kagglehub/datasets/kkhandekar/cheapest-electric-cars/versions/1/Cheapestelectriccars-EVDatabase.csv")

df1.head()

In [ ]:
msno.matrix(df1,figsize=(10,5),fontsize=10)

In [ ]:

# Removijng currency symbols and convert price columns to numeric values
df1['PriceinGermany'] = pd.to_numeric(df1['PriceinGermany'].str.replace('€', '').str.replace(',', ''), errors='coerce')
df1['PriceinUK'] = pd.to_numeric(df1['PriceinUK'].str.replace('£', '').str.replace(',', ''), errors='coerce')

# Removing "km/h" or "km" from relevant columns and convert them to numeric
df1['TopSpeed'] = pd.to_numeric(df1['TopSpeed'].str.replace(' km/h', ''), errors='coerce')
df1['Range'] = pd.to_numeric(df1['Range'].str.replace(' km', ''), errors='coerce')
df1['Efficiency'] = pd.to_numeric(df1['Efficiency'].str.replace(' Wh/km', ''), errors='coerce')
df1['FastChargeSpeed'] = pd.to_numeric(df1['FastChargeSpeed'].str.replace(' km/h', ''), errors='coerce')
# if priceis availale in germany but uk has NaN it will use that price to fill by conbverting german price to uk

df1['PriceinUK'] = df1['PriceinUK'].fillna(df1['PriceinGermany'] * 0.84)
# if both has NaN filling uk price with mean price
df1['PriceinUK'] = df1['PriceinUK'].fillna(df1['PriceinUK'].mean())
#fixing fast charging mising values with mean values

df1['FastChargeSpeed'] = df1['FastChargeSpeed'].fillna(df1['FastChargeSpeed'].mean())

df1.head()


In [ ]:
msno.matrix(df1,figsize=(10,5),fontsize=10)

In [ ]:
df1.info()

**loading dataset 2**

In [ ]:
url = "https://raw.githubusercontent.com/awais345/Electric-vehicles/main/Electric%20Car.csv"
df2 = pd.read_csv(url)
df2.head()

In [ ]:
df2.info()

In [ ]:
msno.matrix(df2,figsize=(10,5),fontsize=10)

**Merging the datasets to create 1 big dataset**

In [ ]:

# we note that 2 columns merged can make name column of df1 from df2
# so 'Name' column in df2 is made by combining 'Brand' and 'Model'
df2['Name'] = df2['Brand'] + " " + df2['Model']

#  Renaming the  df2 columns to match with df1
df2_renamed = df2.rename(columns={
    'AccelSec': 'Acceleration',
    'TopSpeed_KmH': 'TopSpeed',
    'Range_Km': 'Range',
    'Efficiency_WhKm': 'Efficiency',
    'FastCharge_KmH': 'FastChargeSpeed',
    'Seats': 'NumberofSeats',
    'PriceEuro': 'PriceinEuro'
})

# now we have  matching columns in both datsets with equal type of units
# converting the Price in Euro to UK pound ( 1 Euro = 0.84 )
df2_renamed['PriceinUK'] = df2_renamed['PriceinEuro'] * 0.84

#  also Renaming the  df1 columns to match with df1
df2_final = df2_renamed[['Name', 'Acceleration', 'TopSpeed', 'Range', 'Efficiency', 'FastChargeSpeed', 'NumberofSeats', 'PriceinUK']]
print(df2_final.head())
# using only matching columns from df1
df1_final = df1[['Name', 'Acceleration', 'TopSpeed', 'Range', 'Efficiency', 'FastChargeSpeed', 'NumberofSeats', 'PriceinUK']]
print(df1_final.head())
# merging both columns
df = pd.concat([df1_final, df2_final], ignore_index=True)


print(df.info())

In [ ]:
df

# **EDA**

In [ ]:
# checking for the missing values for the merged dataset

msno.matrix(df,figsize=(10,5),fontsize=10)

In [ ]:
sns.heatmap(df.isnull(), cbar=False, cmap='viridis')


In [ ]:
print(df.describe())

In [ ]:
# Distribution of categorical columns
print(df['Name'].value_counts())
print(df['Acceleration'].value_counts())
print(df['FastChargeSpeed'].value_counts())


In [ ]:
# Histogram for numerical columns
numerical_columns = ['TopSpeed', 'Range', 'Efficiency', 'NumberofSeats', 'PriceinUK']
df[numerical_columns].hist(bins=20, figsize=(12, 8))
plt.tight_layout()
plt.show()

# Box plot for PriceinUK
sns.boxplot(x=df['PriceinUK'])
plt.show()


In [ ]:
# Bar plot for 'NumberofSeats'
sns.countplot(x='NumberofSeats', data=df)
plt.show()


In [ ]:
# Pairplot for numerical columns
sns.pairplot(df[numerical_columns])
plt.show()


In [ ]:
# Correlation heatmap
corr = df[numerical_columns].corr()
sns.heatmap(corr, annot=True, cmap='coolwarm')
plt.show()


In [ ]:
sns.relplot(x='Range', y='PriceinUK', data=df)

### **Feature Egineering**

In [ ]:
# removing non numeric units
df['Acceleration'] = df['Acceleration'].str.replace(' sec', '', regex=False).astype(float)

df['FastChargeSpeed'] = pd.to_numeric(df['FastChargeSpeed'], errors='coerce')
# droping the name and target column as the name brand impact is unpredictable
X = df.drop(columns=['Name', 'PriceinUK'])

# seting the target variable
y = df['PriceinUK']



In [ ]:
X = df[['Range', 'TopSpeed', 'Efficiency', 'NumberofSeats']]
y = df['PriceinUK']

model = RandomForestRegressor()
model.fit(X, y)

# Ploting feature importance using RF-regressor
importance = model.feature_importances_
sns.barplot(x=X.columns, y=importance)

In [ ]:
df.info()

In [ ]:
df.columns

In [ ]:
X = df.drop(columns=['Name', 'PriceinUK'])
y = df['PriceinUK']


In [ ]:
# Feature Engineering: SMA, EMA, MACD, ROC
# PriceinUK is oir feature for the indicators

# Calculating Simple Moving Average (SMA)
df['SMA_20'] = df['PriceinUK'].rolling(window=20).mean()  # 20-period SMA

# Calculating Exponential Moving Average (EMA)
df['EMA_20'] = df['PriceinUK'].ewm(span=20, adjust=False).mean()  # 20-period EMA

# Calculating MACD (Moving Average Convergence Divergence)
# Fast EMA (12-period)
df['EMA_12'] = df['PriceinUK'].ewm(span=12, adjust=False).mean()
# Slow EMA (26-period)
df['EMA_26'] = df['PriceinUK'].ewm(span=26, adjust=False).mean()
# MACD Line
df['MACD'] = df['EMA_12'] - df['EMA_26']
# Signal Line (9-period EMA of MACD)
df['Signal_Line'] = df['MACD'].ewm(span=9, adjust=False).mean()


# Calculating Rate of Change (ROC)
# 1-period ROC in percentage
df['ROC'] = df['PriceinUK'].pct_change(periods=1) * 100

# Ploting all indicators in a single plot
plt.figure(figsize=(12, 8))

# Ploting the original PriceinUK
plt.plot(df['PriceinUK'], label='Price in UK', color='blue', alpha=0.7)

# Ploting SMA and EMA
plt.plot(df['SMA_20'], label='SMA 20', color='orange', linestyle='--')
plt.plot(df['EMA_20'], label='EMA 20', color='green', linestyle='-.')

# Ploting MACD and Signal Line
plt.plot(df['MACD'], label='MACD', color='red', linestyle='-')
plt.plot(df['Signal_Line'], label='Signal Line', color='purple', linestyle=':')

# Ploting ROC (Rate of Change)
plt.plot(df['ROC'], label='ROC', color='brown', linestyle='-')

# Setting t he plot title and labels
plt.title('Technical Indicators: SMA, EMA, MACD, and ROC for PriceinUK')
plt.xlabel('Index')
plt.ylabel('Value')
plt.legend()

# Show the plot
plt.tight_layout()
plt.show()


In [ ]:
# filling all nulls
df['Acceleration'].fillna(df['Acceleration'].median(), inplace=True)
df['FastChargeSpeed'].fillna(df['FastChargeSpeed'].median(), inplace=True)
df['SMA_20'].fillna(df['SMA_20'].median(), inplace=True)
df['ROC'].fillna(df['ROC'].median(), inplace=True)
print(df.isnull().sum())


# Random forest

In [ ]:
def random_forest(ts,nst):

  # Spliting nin test train ratio
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

  # Training the Random Forest Model
  rf_model = RandomForestRegressor(n_estimators=nst, random_state=42)
  rf_model.fit(X_train, y_train)

  # Predicting
  y_pred = rf_model.predict(X_test)

  # Evaluating the Model
  mae = mean_absolute_error(y_test, y_pred)
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  rse = r2_score(y_test, y_pred)
  print("\n\n")
  print("-"*60)
  print("-"*60)
  print("Mean Absolute Error (MAE):", mae)
  print("Mean Squared Error (MSE):", mse)
  print("Root Mean Squared Error (RMSE):", rmse)
  print("R-Squared Score (R2SE):", rse)
  print("-"*60)
  print("-"*60)

  # Graph ploting
  plt.figure(figsize=(10, 6))
  plt.scatter(y_test, y_pred
              , alpha=0.5)
  plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
  plt.xlabel('Actual Prices')
  plt.ylabel('Predicted Prices')
  plt.title('Actual Prices vs. Predicted Prices')
  plt.show()
    # Error Distribution Plot
  residuals = y_test - y_pred
  plt.figure(figsize=(10, 6))
  sns.histplot(residuals, kde=True, bins=30, color='blue')
  plt.xlabel('Residuals (Errors)')
  plt.title('Error Distribution')
  plt.show()

  return rf_model




In [ ]:
model = random_forest(ts=0.2,nst=100)

In [ ]:
model2 = random_forest(ts=0.3,nst=150)

In [ ]:
model3 = random_forest(ts=0.3,nst=100)

*predicting the price**

In [ ]:
def predict_car_price(model, feature_order):
    print("Please enter the values for the following features:")

    try:

        top_speed = float(input("Enter the top speed (in km/h): "))
        range_km = float(input("Enter the range (in km): "))
        efficiency = float(input("Enter the efficiency (in Wh/km): "))
        fast_charge_speed = float(input("Enter the fast charge speed (in km/h): "))
        number_of_seats = int(input("Enter the number of seats: "))
        acceleration = float(input("Enter the acceleration (in seconds): "))

        # dictionary for input values
        input_data = {
            'TopSpeed': top_speed,
            'Range': range_km,
            'Efficiency': efficiency,
            'FastChargeSpeed': fast_charge_speed,
            'NumberofSeats': number_of_seats,
            'Acceleration': acceleration
        }


        input_df = pd.DataFrame([input_data])

        # matching the feature
        input_df = input_df[feature_order]

        # error tackling
        print("\nInput Data for Prediction:\n", input_df)

        # Predicting the price
        predicted_price = model.predict(input_df)[0]
        print(f"\nThe predicted price of the car is: €{predicted_price:,.2f}")

    except ValueError as ve:
        print(f"Input Error: {ve}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


In [ ]:
predict_car_price(model3, X.columns)


# **XGBoost**

In [ ]:
def xgboost(df, n_estim,lr,dep,ts):

  # Dropping missing values
  data = df.copy()
  data.dropna(inplace=True)

  # Splitting the data into training and testing datasets
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

  # Initializing and fitting the XGBoost model
  xg_model = xgb.XGBRegressor(objective='reg:squarederror',
                              n_estimators=n_estim,
                              learning_rate=lr,
                              max_depth=dep)
  xg_model.fit(X_train, y_train)

  # Making predictions
  y_pred = xg_model.predict(X_test)

  # Calculate RMSE, MSE, MAE, and R²
  mse = mean_squared_error(y_test, y_pred)
  rmse = np.sqrt(mse)
  mae = mean_absolute_error(y_test, y_pred)
  rse = r2_score(y_test, y_pred)

  # Output the metrics
  print("\n" + "-"*60)
  print("XGBoost Summary")
  print("-"*60)
  print(f"RMSE of the trained model: {rmse:.3f}")
  print(f"MSE of the trained model: {mse:.3f}")
  print(f"MAE of the trained model: {mae:.3f}")
  print(f"R-Squared Error (R²): {rse:.3f}")
  print("-"*60 + "\n")
  # Graph ploting
  plt.figure(figsize=(10, 6))
  plt.scatter(y_test, y_pred
              , alpha=0.5)
  plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
  plt.xlabel('Actual Prices')
  plt.ylabel('Predicted Prices')
  plt.title('Actual Prices vs. Predicted Prices')
  plt.show()
    # Error Distribution Plot
  residuals = y_test - y_pred
  plt.figure(figsize=(10, 6))
  sns.histplot(residuals, kde=True, bins=30, color='blue')
  plt.xlabel('Residuals (Errors)')
  plt.title('Error Distribution')
  plt.show()
  return xg_model

In [ ]:
model =xgboost(df,100,0.01,5,0.2)

In [ ]:
model =xgboost(df,100,0.01,5,0.3)

In [ ]:
model =xgboost(df,100,0.1,5,0.3)

**Predicting price**

In [ ]:
def predict_with_xgboost_interactive(model):
    """
    Function to interactively predict the car price using the trained XGBoost model.
    Returns:
    Predicted price.
    """
    try:
        # Prompt the user to input values for each feature
        print("Please enter the following details for price prediction:\n")

        acceleration = float(input("Acceleration (in seconds, e.g., 5.0): "))
        top_speed = int(input("Top Speed (in km/h, e.g., 220): "))
        car_range = int(input("Range (in km, e.g., 400): "))
        efficiency = int(input("Efficiency (in Wh/km, e.g., 160): "))
        fast_charge_speed = float(input("Fast Charge Speed (in km/h, e.g., 800): "))
        number_of_seats = int(input("Number of Seats (e.g., 5): "))

        # Create a DataFrame with the user inputs
        input_features = {
            'Acceleration': acceleration,
            'TopSpeed': top_speed,
            'Range': car_range,
            'Efficiency': efficiency,
            'FastChargeSpeed': fast_charge_speed,
            'NumberofSeats': number_of_seats
        }
        input_df = pd.DataFrame([input_features])

        # Ensure the column order matches the training data
        input_df = input_df[X.columns]

        # Predict the price directly using the model
        predicted_price = model.predict(input_df)[0]

        print(f"\nPredicted Price (in UK): £{predicted_price:.2f}")
        return predicted_price

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return None


In [ ]:
predict_with_xgboost_interactive(model)


# SVM

In [ ]:
def svr_model(ts, kernel, X, y):
    # Splitting the dataset
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=ts, random_state=42)

    # Training the SVR Model
    svr = SVR(kernel=kernel)
    svr.fit(X_train, y_train)

    # Predicting
    y_pred = svr.predict(X_test)

    # Evaluating the Model
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mae = mean_absolute_error(y_test, y_pred)
    rse = r2_score(y_test, y_pred)

    print("-" * 60)
    print("SVR Summary")
    print("-" * 60)
    print("Mean Squared Error (MSE):", mse)
    print("Root Mean Squared Error (RMSE):", rmse)
    print("Mean Absolute Error (MAE):", mae)
    print("R-Squared Error (R2SE):", rse)
    print("-" * 60)

    # Plotting Actual vs Predicted Values
    plt.figure(figsize=(10, 6))
    plt.scatter(y_test, y_pred, alpha=0.5)
    plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
    plt.xlabel('Actual Prices')
    plt.ylabel('Predicted Prices')
    plt.title('Actual Prices vs. Predicted Prices (SVR)')
    plt.show()

    # Error Distribution Plot
    residuals = y_test - y_pred
    plt.figure(figsize=(10, 6))
    sns.histplot(residuals, kde=True, bins=30, color='blue')
    plt.xlabel('Residuals (Errors)')
    plt.title('Error Distribution')
    plt.show()

    # Save the model to a file
    with open('svr_model.pkl', 'wb') as f:
        pickle.dump(svr, f)

    print("Model saved as 'svr_model.pkl'")

    return svr


In [ ]:
# Imputing missing values with the mean for SVR model
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X)

In [ ]:
model = svr_model(ts=0.2, kernel='rbf', X=X_imputed, y=y)


In [ ]:
model = svr_model(ts=0.3, kernel='rbf', X=X_imputed, y=y)

In [ ]:
model = svr_model(ts=0.35, kernel='rbf', X=X_imputed, y=y)

**Predicitng the prices**

In [ ]:
def predict_car_price_svr(model_file):
    # Load the saved model
    with open(model_file, 'rb') as f:
        svr_model = pickle.load(f)

    print("Please enter the values for the following features:")

    # Taking user input for each feature
    top_speed = float(input("Enter the top speed (in km/h): "))
    range_km = float(input("Enter the range (in km): "))
    efficiency = float(input("Enter the efficiency (in Wh/km): "))
    fast_charge_speed = float(input("Enter the fast charge speed (in km/h): "))
    number_of_seats = int(input("Enter the number of seats: "))
    acceleration = float(input("Enter the acceleration (in seconds): "))

    # Creating a DataFrame for the input values
    input_data = pd.DataFrame({
        'TopSpeed': [top_speed],
        'Range': [range_km],
        'Efficiency': [efficiency],
        'FastChargeSpeed': [fast_charge_speed],
        'NumberofSeats': [number_of_seats],
        'Acceleration': [acceleration]
    })

    # Predicting the price using the loaded model
    predicted_price = svr_model.predict(input_data)[0]
    print(f"\nThe predicted price of the car is: €{predicted_price:,.2f}")


In [ ]:
predict_car_price_svr('svr_model.pkl')


# Linear Regression function

In [ ]:
def train_linear_regression_model(X, y, randoms,ts):
    """
    Function to train a Linear Regression model with missing value imputation,
    and plot actual vs predicted values and error distribution.
    """
    try:
        # Impute missing values using the mean
        imputer = SimpleImputer(strategy='mean')
        X_imputed = imputer.fit_transform(X)

        # Split the data into training and testing sets
        X_train, X_test, y_train, y_test = train_test_split(X_imputed, y, test_size=ts, random_state=randoms)

        # Train the Linear Regression model
        model = LinearRegression()
        model.fit(X_train, y_train)

        # Predict on the test set
        y_pred = model.predict(X_test)

        # Calculate RMSE, MSE, and MAE
        rmse = np.sqrt(mean_squared_error(y_test, y_pred))
        mse = mean_squared_error(y_test, y_pred)
        mae = mean_absolute_error(y_test, y_pred)
        rse = r2_score(y_test, y_pred)

        # Output the metrics
        print("\n\n")
        print("-" * 60)
        print("Linear Regression Summary")
        print("-" * 60)
        print(f"RMSE of the trained model: {rmse:.3f}")
        print(f"MSE of the trained model: {mse:.3f}")
        print(f"MAE of the trained model: {mae:.3f}")
        print(f"R-Squared Error (R2SE): {rse:.3f}")
        print("-" * 60)
        print("\n\n")

        # Plotting Actual vs Predicted Values
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5)
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
        plt.xlabel('Actual Prices')
        plt.ylabel('Predicted Prices')
        plt.title('Actual Prices vs. Predicted Prices (Linear Regression)')
        plt.grid(True)
        plt.show()

        # Error Distribution Plot
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True, bins=30, color='blue')
        plt.xlabel('Residuals (Errors)')
        plt.title('Error Distribution')
        plt.grid(True)
        plt.show()

        # Return the model and metrics
        return model

    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [ ]:
model = train_linear_regression_model(X, y,42,0.2)

In [ ]:

model2 = train_linear_regression_model(X, y,42,0.3)

In [ ]:
model3 = train_linear_regression_model(X, y,1,0.3)

**Price prediction of  linear regression model**

In [ ]:
def predict_with_linear_regression_interactive(model):
    """
    the function asks the user to input values and returns the predicted price.

    parameters:
     model: saved Linear Regression model.

    returns:
     predicted price.
    """
    try:
        # user input values for each feature
        print("Please enter the following details for price prediction:\n")

        acceleration = float(input("Acceleration (in seconds): "))
        top_speed = int(input("Top Speed (in km/h): "))
        car_range = int(input("Range (in km): "))
        efficiency = int(input("Efficiency (in Wh/km "))
        fast_charge_speed = float(input("Fast Charge Speed (in km/h): "))
        number_of_seats = int(input("Number of Seats (e.g., 5): "))

        # Creating a DataFrame with the user inputs
        input_features = {
            'Acceleration': acceleration,
            'TopSpeed': top_speed,
            'Range': car_range,
            'Efficiency': efficiency,
            'FastChargeSpeed': fast_charge_speed,
            'NumberofSeats': number_of_seats
        }
        input_df = pd.DataFrame([input_features])


        input_df = input_df[X.columns]

        # Predicting price
        predicted_price = model.predict(input_df)[0]

        print(f"\nPredicted Price (in UK): £{predicted_price:.2f}")
        return predicted_price

    except Exception as e:
        print(f"An error occurred during prediction: {e}")
        return None


In [ ]:
predict_with_linear_regression_interactive(model3)

# Time Series Transformer

addional preprocessing for  regression transoformer

In [ ]:
# Checking for NaN  valu
print("Checking NaN values in X:")
print(X.isnull().sum())  # Checkinh NaNs in each column

print("\nChecking Inf values in X:")
print(np.isinf(X).sum())  # Checking Infinite values in each column

print("\nChecking NaN or Inf in target variable (y):")
print(pd.Series(y).isnull().sum())
print(pd.Series(y).apply(np.isinf).sum())

# Drop or Fill NaNs and Infs
X = X.replace([np.inf, -np.inf], np.nan)  # Replace Infs with NaN
X = X.fillna(X.mean())  # Fill NaNs with column means

y = pd.Series(y).replace([np.inf, -np.inf], np.nan)
y = y.fillna(y.mean())

# Verify again
print("\nAfter Cleaning:")
print(X.isnull().sum())
print(pd.Series(y).isnull().sum())


In [ ]:
# Verify Dataset
if np.isnan(X).any().any() or np.isnan(y).any():
    raise ValueError("NaN values are still present in the dataset after cleaning!")
else:
    print("Dataset is clean. Proceeding to training...")


In [ ]:
# Transformer Regressor Model
class TransformerRegressor(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_heads, num_layers, output_dim):
        super(TransformerRegressor, self).__init__()

        self.input_projection = nn.Linear(input_dim, hidden_dim)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=hidden_dim,
            nhead=num_heads,
            dim_feedforward=hidden_dim * 4,
            dropout=0.1,
            activation='relu',
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.fc_out = nn.Linear(hidden_dim, output_dim)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.input_projection(x)
        x = self.transformer_encoder(x)
        x = x.mean(dim=1)
        x = self.fc_out(x)
        return x

# Training Function
def train_transformer(X, y, rand ,epochs, lr,ts):
    try:
        y = y.values if hasattr(y, 'values') else np.array(y)

        # Data Preprocessing scaling
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # Checking for NaNs after scaling although it was done in addional preprocessing in abobe cells
        if np.isnan(X_scaled).any() or np.isnan(y).any():
            raise ValueError("NaN values detected in the dataset after scaling. Check input data.")

        X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=ts, random_state=rand)

        # Convert to PyTorch tensors
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(1)
        y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32).unsqueeze(1)
        y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

        # Define model, loss, and optimizer
        input_dim = X_train.shape[1]
        hidden_dim = 64
        num_heads = 4
        num_layers = 2
        output_dim = 1

        model = TransformerRegressor(input_dim, hidden_dim, num_heads, num_layers, output_dim)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=lr)

        # Training Loop
        train_losses = []
        for epoch in range(epochs):
            model.train()
            optimizer.zero_grad()
            output = model(X_train_tensor)
            loss = criterion(output.squeeze(), y_train_tensor.squeeze())

            # Check for NaN in Loss
            if torch.isnan(loss):
                raise ValueError("NaN loss detected during training. Adjust learning rate or model architecture.")

            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
            optimizer.step()
            train_losses.append(loss.item())

            if epoch % 10 == 0:
                print(f'Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}')

        # Plot Training Loss Curve
        plt.figure(figsize=(10, 6))
        plt.plot(train_losses, label='Training Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.title('Training Loss Curve')
        plt.legend()
        plt.show()

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test_tensor).squeeze().numpy()

        mse = mean_squared_error(y_test, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_test, y_pred)
        r2 = r2_score(y_test, y_pred)

        print("\n" + "-" * 60)
        print("Transformer Regressor Summary")
        print("-" * 60)
        print(f"Mean Squared Error (MSE): {mse:.3f}")
        print(f"Root Mean Squared Error (RMSE): {rmse:.3f}")
        print(f"Mean Absolute Error (MAE): {mae:.3f}")
        print(f"R-Squared Error (R²): {r2:.3f}")
        print("-" * 60)

        # Plot Actual vs. Predicted Values
        plt.figure(figsize=(10, 6))
        plt.scatter(y_test, y_pred, alpha=0.5, color='b')
        plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], linestyle='--', color='red')
        plt.xlabel('Actual Prices')
        plt.ylabel('Predicted Prices')
        plt.title('Actual Prices vs. Predicted Prices (Transformer Regressor)')
        plt.show()

        # Plot Error Distribution
        residuals = y_test - y_pred
        plt.figure(figsize=(10, 6))
        sns.histplot(residuals, kde=True, bins=30, color='blue')
        plt.xlabel('Residuals (Errors)')
        plt.title('Error Distribution')
        plt.show()

        return model

    except Exception as e:
        print(f"An error occurred: {e}")
        return None


In [ ]:
model = train_transformer(X, y, rand=1, epochs=50, lr=0.001,ts=0.2)


In [ ]:
model2 = train_transformer(X, y, rand=42, epochs=100, lr=0.001,ts=0.3)


In [ ]:
model3 = train_transformer(X, y, rand=42, epochs=150, lr=0.001,ts=0.2)


**Predicting price**

In [ ]:
def predict_with_transformer_interactive(model, scaler,device='cpu'):
    """
    Function to interactively predict the car price using the trained Transformer model.
    Returns:
    Predicted price.
    """
    # Set model to evaluation mode
    model.eval()
    model.to(device)

    # User input for each feature
    print("Please enter the following details for price prediction:\n")

    acceleration = float(input("Acceleration (in seconds): "))
    top_speed = int(input("Top Speed (in km/h): "))
    car_range = int(input("Range (in km): "))
    efficiency = int(input("Efficiency (in Wh/km): "))
    fast_charge_speed = float(input("Fast Charge Speed (in km/h): "))
    number_of_seats = int(input("Number of Seats (e.g., 5): "))

    # Creating a DataFrame with the user inputs
    input_features = {
        'Acceleration': acceleration,
        'TopSpeed': top_speed,
        'Range': car_range,
        'Efficiency': efficiency,
        'FastChargeSpeed': fast_charge_speed,
        'NumberofSeats': number_of_seats
    }
    input_df = pd.DataFrame([input_features])

    # Ensure the feature order matches the training data
    input_df = input_df[X.columns]

    # Normalize the input using the same scaler used during training
    input_scaled = scaler.transform(input_df)

    # Converting to a PyTorch tensor
    input_tensor = torch.tensor(input_scaled, dtype=torch.float32).unsqueeze(1).to(device)

    # prediction
    with torch.no_grad():
        predicted_price = model(input_tensor).squeeze().item()

    print(f"\nPredicted Price (in UK): £{predicted_price:.2f}")
    return predicted_price


In [ ]:
scaler = StandardScaler()
scaler.fit(X)
predicted_price = predict_with_transformer_interactive(model2,scaler=scaler, device='cpu')
